# Imports

Check we are using python 3.7:

In [4]:
import sys
print(sys.version)

3.7.11 (default, Jul 27 2021, 09:42:29) [MSC v.1916 64 bit (AMD64)]


In [5]:
import os
os.getcwd()

'C:\\Users\\jahutter\\Documents\\Programming\\Thesis code\\CTAB-GAN-main'

Imports and package version checks:

In [ ]:
# AWS Sagemaker gbvCHANGES:
! pip install numpy==1.21.0 #or use conda install -y numpy==1.21.0
! pip install pandas==1.2.4
! pip install torch==1.9.1
! pip install dython==0.6.4.post1
! pip install scipy==1.4.1

In [3]:
import torch
print(torch.cuda.is_available()) # check if CUDA available
print(torch.cuda.device_count()) # should be 1 GPU
print(torch.cuda.current_device()) # which device is used by PyTorch: should be 0
print(torch.cuda.device(0))
print(torch.cuda.get_device_name(0))

False
0


AssertionError: Torch not compiled with CUDA enabled

In [4]:
from model.ctabgan import CTABGAN
import numpy as np
print("Numpy version should be 1.21.0, it is", np.__version__) # np 1.19.5 is okay too I think
import torch
print("Torch version should be 1.9.1, it is", torch.__version__) # 1.9.1+cpu runs only on CPU
import pandas as pd
print("Pandas version should be 1.2.4, it is", pd.__version__) # 0.24.0 might work also???
import sklearn
print("Sklearn version should be 0.24.1, it is", sklearn.__version__)
import dython
print("Dython version should be 0.6.4.post1, it is", dython.__version__)
import scipy
print("Scipy version should be 1.4.1, it is", scipy.__version__)
import glob
import json
import time
pd.set_option('display.max_columns', None) # to see all cols when viewing db

Numpy version should be 1.21.0, it is 1.21.0
Torch version should be 1.9.1, it is 1.9.1+cpu
Pandas version should be 1.2.4, it is 1.2.4
Sklearn version should be 0.24.1, it is 0.24.1
Dython version should be 0.6.4.post1, it is 0.6.4.post1
Scipy version should be 1.4.1, it is 1.4.1


# Selecting and loading real dataset

Loading the datasets & lists of column types:

In [2]:
num_exp =  3
num_epoch =   10 #  150 as in paper
dataset_name = "Census" # "Sachs" #"Asia" # "Adult" #     "Sachs2" #  "Cancer2" #  "Cancer" #  "Intrusion" # 
fake_file_root = "../Fake_Datasets"

if dataset_name in ["Adult", "Census"]: # "Asia", "Sachs"
    real_file_root = "../Real_Datasets/"
else:
    real_file_root = "../BN_Datasets/"
real_path = real_file_root + dataset_name+"/"+dataset_name
#real_path = real_file_root +dataset_name

with open(real_path+"_meta.json") as json_file:
    meta = json.load(json_file)

target = meta["target"]
categorical_cols = meta["categorical_cols"]
categorical_cols.append(target)
mixed_cols = meta["mixed_cols"]
probl_type = {"Classification": target}
log_cols = meta["log_cols"]
integer_cols = meta["numerical_cols"]

real_path = real_path +".csv"
real_data = pd.read_csv(real_path)

real_data.head()

,age,work_class,industry,occupation,education,wage,enroll_in_edu,marital_status,major_industry,major_occupation,race,hispanic_origin,gender,union_member,unemployment_reason,employment_status,capital_gains,capital_loss,stock_dividends,tax_filer_status,reg_prev_residence,state_prev_residence,household_status,household_summary,migr_code_msa,migr_code_reg,migr_code_in_reg,migr_same,migr_sun,n_pers_employer,fam_under_18,country_father,country_mother,country_self,citizenship,own_business,vet_qva,vet_ben,weeks_work,year,tot_pers_income
0,41,4,33,26,12,1500,2,2,19,0,4,0,0,0,3,1,0,0,0,2,3,36,33,7,0,0,0,1,0,1,4,40,40,40,4,0,1,2,51,95,0
1,15,3,0,0,6,0,2,4,14,6,4,0,1,1,3,0,0,0,0,4,3,36,8,2,7,6,7,2,2,0,0,40,40,40,4,0,1,2,0,94,0
2,50,6,4,2,14,0,2,2,4,2,4,0,1,1,3,1,0,0,0,2,3,36,18,4,0,0,0,1,0,1,4,40,40,40,4,0,1,2,52,95,1
3,3,3,0,0,10,0,2,4,14,6,4,0,0,1,3,0,0,0,0,4,3,36,8,2,7,6,7,2,2,0,2,40,40,40,4,0,1,0,0,94,0
4,14,3,0,0,10,0,2,4,14,6,4,6,0,1,3,0,0,0,0,4,3,36,8,2,0,0,0,1,0,0,1,40,40,40,4,0,1,0,0,95,0


# Training model & generating synthetic data

In [3]:
times = []

print("Running ", num_exp, "experiment(s) for ", dataset_name, "with ", num_epoch, " epochs")
synthesizer =  CTABGAN(raw_csv_path = real_path,
                 test_ratio = 0.20, # set 20% apart as test set for evaluation, splitting with random_state = rs
                 categorical_columns = categorical_cols, 
                 log_columns = log_cols,
                 mixed_columns= mixed_cols,
                 integer_columns = integer_cols,
                 problem_type= probl_type,
                 epochs = num_epoch)

for i in list(range(1,num_exp+1)):
    print("PERFORMING EXPERIMENT", i)
    start_time = time.time()
    synthesizer.fit()
    syn = synthesizer.generate_samples()
    syn.to_csv(fake_file_root+"/"+dataset_name+"/"+ dataset_name+"_fake_ctab_"+str(num_epoch)+"epochs_"+str(i)+".csv", index= False)
    end_time = time.time()
    times.append(end_time-start_time)
pd.DataFrame(times, columns=["Comp.times"]).to_csv("../Evaluation/Comp_time/times_"+dataset_name+"_ctab_"+str(num_epoch)+"epochs.csv", index= False)


PERFORMING EXPERIMENT 1
Fit function of CTABGAN called
Fit function of ctabgan synthesizer called



  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

Epoch: 0


 10%|███████▉                                                                       | 1/10 [27:49<4:10:28, 1669.79s/it]

Epoch: 1


...

Epoch: 8


 90%|███████████████████████████████████████████████████████████████████████        | 9/10 [4:30:04<30:02, 1802.71s/it]

Epoch: 9


100%|██████████████████████████████████████████████████████████████████████████████| 10/10 [5:00:18<00:00, 1801.84s/it]


Finished training in 18061.879078626633  seconds.
Finished  3 experiment(s) for  Census with  10  epochs
